In [1]:
# setting up imports
import xmltodict as xd
import pvl
import plio

from plio.utils.utils import find_in_dict
from pvl._collections import OrderedMultiDict

In [51]:
with open("Data/example.xml") as file:
    dict = xd.parse(file.read())  

# Set the encoder for dumping the pvl label
encoder = pvl.encoder.IsisCubeLabelEncoder

############ Get the instrument label ############

instrument_name = find_in_dict(dict, "Instrument_Host_Name")
if instrument_name != 'TRACE GAS ORBITER':
    raise Exception(instrument_name, " is not a supported mission.")

# Get all the things
Instrument_host_Id = find_in_dict(dict, "Instrument_Host_Id")
target_name = find_in_dict(dict, "Target")['#text']
start_time = find_in_dict(dict, "OnboardImageAcquisitionTime")["#text"]
exposure_duration = find_in_dict(dict, "PEHK_HEADER")["@Exposure_Time"]

# create the pvl object
module = pvl.PVLObject([
    ('Instrument', pvl.PVLGroup([
        ('SpacecraftName', instrument_name),
        ('InstrumentId', Instrument_host_Id),
        ('TargetName', target_name),
        ('StartTime', start_time),
        ('ExposureDuration', exposure_duration)
        ])
     )
])

# dump the pvl object into a string
instrument_lbl = pvl.dumps(module, cls=encoder).decode("utf-8")

############ Get the Core label ############
base = find_in_dict(dict, "value_offset")
multiplier = find_in_dict(dict, "scaling_factor")
skip = 0

try: 
    lines = samples = find_in_dict(dict, 'Axis_Array')[1]['elements']
except:
    lines = 1

try: 
    samples = find_in_dict(dict, 'Axis_Array')[0]['elements']
except:
    samples = 1
    
bands = 1  # all images consist of one band for CaSSIS
byte_order = "Lsb"

idl_data_type = find_in_dict(dict, 'idl_data_type')

# convert from IDL data types
if idl_data_type is 2:
    data_type = "SignedWord"
elif idl_data_type is 12:
    data_type = "UnsignedWord"
else: 
    data_type = find_in_dict(dict, 'data_type')

# create the pvl object
module = pvl.PVLModule([
    ('Core', pvl.PVLObject([
        ('Skip', skip),
        ('Dimenstions', pvl.PVLGroup([
                        ('Samples', samples),
                        ('Lines', lines),
                        ('Bands', bands),     
                        ])),
        ('Pixels', pvl.PVLGroup([
                    ('Type', data_type),
                    ('ByteOrder', byte_order),
                    ('Base', base),
                    ('Multiplier', multiplier),
                    ]))
        ])
     )
])

# dump the pvl object into a string
core_lbl = pvl.dumps(module, cls=encoder).decode("utf-8")

lbl = core_lbl + '\n\n' + instrument_lbl

print(lbl)

Object = Core
  Skip = 0
  Group = Dimenstions
    Samples = "1536"
    Lines = "255"
    Bands = 1
  End_Group
  Group = Pixels
    Type = SignedWord
    ByteOrder = Lsb
    Base = "0.00"
    Multiplier = "1.00"
  End_Group
End_Object
End

Group = Instrument
  SpacecraftName = "TRACE GAS ORBITER"
  InstrumentId = TGO
  TargetName = "Starfield (HD 13591)"
  StartTime = "2016-04-07T09:44:04.839"
  ExposureDuration = "1.008e-003"
End_Group
End
